In [1]:
import pickle
import cv2
import os
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import ruptures as rpt  # our package
from PIL import Image

def determine_drive_state_changed(timeseries: list, n_bkps: int = 3) -> list:
    timeseries = np.array(timeseries)
    timeseries = timeseries / np.linalg.norm(timeseries)
    algo = rpt.KernelCPD(kernel="rbf").fit(timeseries)

    # Detect breakpoints
    breakpoints = algo.predict(n_bkps=n_bkps)
    driver_state_changed = [any(i >= bp for bp in breakpoints) for i in range(len(timeseries))]
    assert len(driver_state_changed) == len(timeseries)
    return driver_state_changed


## Calculate driver change

In [2]:
# Load optical flow data
data = torch.load('resources/optical-flow/optical_flow.pkl', weights_only=False)
records = []
for video_name, frames in data.items():
    if video_name == 'video_00013.mp4':
        video_name = 'video_0013.mp4'
    for frame_data in frames:
        records.append({
            "filename": video_name,
            "frame": frame_data["frame"],
            "score": frame_data["score"]
        })
df_of = pd.DataFrame(records)
df_of['video'] = df_of['filename'].apply(lambda x: x.split('.')[0])


# Add driver change
groups = []
for video, group in df_of.groupby('video'):

    ts = group['score'].bfill().reset_index(drop=True)
    group['change_bkp4'] = determine_drive_state_changed(ts, n_bkps=4)

    groups.append(group)
df_of = pd.concat(groups).reset_index(drop=True)

## Create submission

In [3]:
# Load submission structure
df = pd.read_csv('./submissions/results_driverstate_alltracks_bboxsizes.csv')
df['frame'] = df['ID'].apply(lambda x: int(x.split('_')[-1]) )
df['video'] = df['ID'].apply(lambda x: '_'.join(x.split('_')[:2]) )


with open('./resources/annotations_public.pkl', 'rb') as f:
    anns = pickle.load(f)

data = []
for video, video_data in anns.items():
    for frame, frame_data in video_data.items():
        data.append({
            'ID': f"{video}_{frame}",
            'video': video,
            'frame': frame,
        })

df = pd.DataFrame(data)
df = pd.merge(left=df, right=df_of, how='left', on=['video', 'frame'])
df = df.groupby('video').apply(lambda x: x.ffill())
df = df.reset_index(drop=True)

df_bkp4 = df[['ID', 'change_bkp4']].reset_index(drop=True)
df_bkp4 = df_bkp4.rename({'change_bkp4': 'Driver_State_Changed'}, axis=1)
df_bkp4.to_csv('./submissions/results_driverstate_alltracks_opticalflow.csv')

/tmp/ipykernel_65922/2425918050.py:21: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('video').apply(lambda x: x.ffill())
